In [3]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [4]:
bucket = 'a.test'
prefix = 'spot-price-forecast-poc/DEMO-deepar'

sagemaker_session = sagemaker.Session()
role = get_execution_role()

s3_data_path = "{}/{}/data".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

In [5]:
from sagemaker.amazon.amazon_estimator import get_image_uri
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

In [6]:
freq = '5min'
prediction_length = 400
context_length = 600

In [7]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    base_job_name='DEMO-deepar',
    output_path="s3://" + s3_output_path
)

In [8]:
hyperparameters = {
    "time_freq": freq,
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_cells": "40",
    "num_layers": "3",
    "likelihood": "gaussian",
    "epochs": "20",
    "mini_batch_size": "32",
    "learning_rate": "0.001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10"
}

In [9]:
estimator.set_hyperparameters(**hyperparameters)

In [10]:
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels)

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Could not assume role arn:aws:iam::782037118710:role/aws-reserved/sso.amazonaws.com/ap-southeast-2/AWSReservedSSO_aws-savvybi-advanced-dev_4397105300386060. Please ensure that the role exists and allows principal 'sagemaker.amazonaws.com' to assume the role.